In [1]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 7.3 MB/s eta 0:00:00


In [5]:
from google.colab import files
import pandas as pd
import csv
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD

# Prompt for file upload
uploaded = files.upload()

# Read the uploaded CSV file into a pandas DataFrame
for filename in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=filename, length=len(uploaded[filename])))
    df = pd.read_csv(filename)

Saving Seri.csv to Seri.csv
User uploaded file "Seri.csv" with length 884978 bytes


In [40]:
# GPS collar
import csv
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD
from datetime import datetime

# Define namespaces
FOO = Namespace("https://w3id.org/def/foo#")
SOSA = Namespace("http://www.w3.org/ns/sosa/")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")

# Create a new graph
g = Graph()
# Bind namespaces
g.bind("foo", FOO)
g.bind("sosa", SOSA)
g.bind("geo", GEO)

# Path to your CSV file
csv_file_path = "Seri.csv"  # Adjust the path accordingly

# Read and process data
with open(csv_file_path, newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')

    # Print headers for debugging
    print("Headers:", reader.fieldnames)

    for row in reader:
        identifier = row.get('ID', '').strip()
         # local_date = row.get('LocalDate', '').strip()
        local_date = datetime.strptime(row.get('LocalDate', '').strip(), '%Y-%m-%d').isoformat() if row.get('LocalDate', '').strip() and '00' not in row.get('LocalDate', '') else None
        # local_time = row.get('LocalTime', '').strip()
        local_time = (datetime.strptime(row.get('LocalTime', '').strip(), '%H:%M:%S').time().isoformat() if ':' in row.get('LocalTime', '').strip() and 'AM' not in row.get('LocalTime', '').strip() and 'PM' not in row.get('LocalTime', '').strip() else datetime.strptime(row.get('LocalTime', '').strip().replace('AM', ' AM').replace('PM', ' PM'), '%I:%M:%S %p').time().isoformat()) if row.get('LocalTime', '').strip() else None
        # gmt_date = row.get('GMTDate', '').strip()
        gmt_date = (datetime.strptime(row.get('GMTDate', '').strip(), '%Y-%m-%d').isoformat() if '-' in row.get('GMTDate', '').strip() else datetime.strptime(row.get('GMTDate', '').strip(), '%m/%d/%Y').isoformat()) if row.get('GMTDate', '').strip() and '00' not in row.get('GMTDate', '') else None
        # gmt_time = row.get('GMTTime', '').strip()
        gmt_time = (datetime.strptime(row.get('GMTTime', '').strip(), '%H:%M:%S').time().isoformat() if ':' in row.get('GMTTime', '').strip() and 'AM' not in row.get('GMTTime', '').strip() and 'PM' not in row.get('GMTTime', '').strip() else datetime.strptime(row.get('GMTTime', '').strip().replace('AM', ' AM').replace('PM', ' PM'), '%I:%M:%S %p').time().isoformat()) if row.get('GMTTime', '').strip() else None
        lat = row.get('lat', '').strip()
        long = row.get('long', '').strip()
        temperature = row.get('Temperature', '').strip()
        extTemp = row.get('ExtTemp', '').strip()
        activity = row.get('Activity', '').strip()
        speed = row.get('Speed', '').strip()
        direction = row.get('Direction', '').strip()
        altitude = row.get('Altitude', '').strip()
        cov = row.get('Cov', '').strip()
        hdop = row.get('HDOP', '').strip()
        distance = row.get('Distance', '').strip()
        count = row.get('Count', '').strip()
        # Create observation URI
        observation = URIRef(f"https://w3id.org/def/foo#{identifier}")
        g.add((observation, RDF.type, FOO.gPSObservation))
        g.add((observation, FOO.id, Literal(identifier, datatype=XSD.string)))
        g.add((observation, FOO.madeBySensor, FOO.seriGPS))  # Link sensor to observation
        g.add((observation, FOO.hasFeatureOfInterest, FOO.seri))
        # Add observation properties
        g.add((observation, FOO.localDate, Literal(local_date, datatype=XSD.date)))
        g.add((observation, FOO.localTime, Literal(local_time, datatype=XSD.time)))
        g.add((observation, FOO.gMTDate, Literal(gmt_date, datatype=XSD.date)))
        g.add((observation, FOO.gMTTime, Literal(gmt_time, datatype=XSD.time)))
        g.add((observation, GEO.latitude, Literal(lat, datatype=XSD.double)))
        g.add((observation, GEO.longitude, Literal(long, datatype=XSD.double)))

        # Add observable properties if available
        if temperature:
            g.add((observation, FOO.temperature, Literal(temperature, datatype=XSD.double)))
        if extTemp:
            g.add((observation, FOO.extTemp, Literal(temperature, datatype=XSD.double)))
        if activity:
            g.add((observation, FOO.activity, Literal(temperature, datatype=XSD.double)))
        if speed:
            g.add((observation, FOO.speed, Literal(speed, datatype=XSD.double)))
        if direction:
            g.add((observation, FOO.direction, Literal(direction, datatype=XSD.integer)))
        if altitude:
            g.add((observation, FOO.altitude, Literal(altitude, datatype=XSD.string)))
        if cov:
            g.add((observation, FOO.cov, Literal(cov, datatype=XSD.double)))
        if hdop:
            g.add((observation, FOO.hDOP, Literal(hdop, datatype=XSD.double)))
        if distance:
            g.add((observation, FOO.distance, Literal(distance, datatype=XSD.double)))
        if count:
            g.add((observation, FOO.cOUNT, Literal(count, datatype=XSD.integer)))

# Serialize the graph to a file
output_file = "Seri_rdf_graph.ttl"
g.serialize(destination=output_file, format="turtle")

print(f"Knowledge graph has been serialized to {output_file}")



Headers: ['ID', 'LocalDate', 'LocalTime', 'GMTDate', 'GMTTime', 'lat', 'long', 'Temperature', 'ExtTemp', 'Activity', 'Speed', 'Direction', 'Cov', 'HDOP', 'Distance', 'Count']
Knowledge graph has been serialized to Seri_rdf_graph.ttl


In [ ]:
# Download the file
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>